# Evaluate the final ranking prediction versus the real results

In [1]:
import re
import sqlite3
import pandas as pd
import numpy as np
from textblob import TextBlob
from collections import Counter
from random import randint

## Aux functions

In [2]:
def get_tweet_sentiment(tweet):
    """
    Utility function to classify sentiment of passed tweet
    using textblob's sentiment method
    """

    # create TextBlob object of passed tweet text
    analysis = TextBlob(clean_tweet(tweet['tweetText']))
    
    # set sentiment
    if analysis.sentiment.polarity > 0:
        return 'positive'
    elif analysis.sentiment.polarity == 0:
        return 'neutral'
    else:
        return 'negative'

In [3]:
def clean_tweet(tweet):
    '''
    Utility function to clean tweet text by removing links, special characters
    using simple regex statements.
    '''
    return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split())

## Setup

In [4]:
# Setup sqlite
sqlite_file = 'db_friday_and_final.db'

# Connect to the database sqlite file
connection = sqlite3.connect(sqlite_file)
db = connection.cursor()

In [5]:
# set country hashtags - semifinal 1
all_hashtags = ['SWE', 'GEO', 'AUS', 'ALB', 'BEL', 'MNE', 'FIN', 'AZE', 'POR',\
                'POL', 'MDA', 'ISL', 'CZE', 'CYP', 'ARM', 'SLO', 'LAT', 'GRE',\
                'AUT', 'BLR', 'DEN', 'EST', 'MKD', 'HUN', 'IRL', 'ISR', 'LTU',\
                'NOR', 'ROM', 'SMR', 'SRB', 'SUI', 'NED', 'CRO', 'BUL', 'MLT',\
                'ITA', 'FRA', 'ESP', 'GBR', 'UKR', 'GER']

hashtags_semi1 = ['SWE', 'GEO', 'AUS', 'ALB', 'BEL', 'MNE', 'FIN', 'AZE', 'POR', 'GRE',\
                    'POL', 'MDA', 'ISL', 'CZE', 'CYP', 'ARM', 'SLO', 'LAT']

# set country hashtags - semifinal 2
hashtags_semi2 = ['AUT', 'BLR', 'DEN', 'EST', 'MKD', 'HUN', 'IRL', 'ISR', 'LTU', 'MLT', \
                    'NOR', 'ROM', 'SMR', 'SRB', 'SUI', 'NED', 'CRO', 'BUL']

# set country hashtags - final
hashtags_final = ['ARM', 'AZE', 'ITA', 'MDA', 'POL', 'POR', 'UKR', 'AUS', 'BEL', 'CYP', 'FRA',\
                  'GER', 'GRE', 'ESP', 'GBR', 'SWE', 'BUL', 'BLR', 'CRO', 'HUN', 'DEN',\
                  'ISR', 'ROM', 'NOR', 'NED', 'AUT']

hashtags = hashtags_final
print(len(hashtags))

26


## Count tweets and analyze sentiment

In [6]:
# read ALL tweets in english from db, evaluate sentiment, and count - SEMI 1
all_sentiments = []
for country in hashtags:

    # get tweets from DB
    country_tweets = pd.read_sql_query("SELECT * FROM TweetsRaw WHERE language='en' AND tweetText LIKE '%#{}%'".format(country), connection)

    # count number of sentiments
    sentiments_count = Counter(country_tweets.apply(get_tweet_sentiment, axis=1))
    
    # append country to list
    all_sentiments.append({'country': country, \
                           'positive': sentiments_count['positive'],\
                           'neutral': sentiments_count['neutral'],\
                           'negative': sentiments_count['negative']
                          })

In [7]:
# read all tweets (to just count)
all_tweet_counts = []
for country in hashtags:

    # get tweet count from DB
    db.execute("SELECT COUNT(*) AS count FROM TweetsRaw WHERE tweetText LIKE '%#{}%'".format(country))
    country_tweet_count = db.fetchone()[0]
    
    # append country to list
    all_tweet_counts.append({'country': country, \
                           'count': country_tweet_count
                          })

In [8]:
# transform to pandas dataframe from sentiments list
results = pd.DataFrame(all_sentiments)
results = results.set_index(['country'])

# add total tweet count
results['tweets'] = [tc['count'] for tc in all_tweet_counts]

# add percentages of features over the totals
results['positive_perc'] = results['positive'] / results['positive'].sum()
results['negative_perc'] = results['negative'] / results['negative'].sum()
results['neutral_perc'] = results['neutral'] / results['neutral'].sum()
results['tweets_perc'] = results['tweets'] / results['tweets'].sum()

# create log features
results['negative_log'] = np.log(1 + results['negative_perc'])
results['neutral_log'] = np.log(1 + results['neutral_perc'])
results['positive_log'] = np.log(1 + results['positive_perc'])
results['tweets_log'] = np.log(1 + results['tweets_perc'])

In [9]:
# Apply model coeficients to data and compute 
features = ['negative_log', 'neutral_log', 'positive_log', 'tweets_log']
model_coefs = np.array([-8.93416811e-03, -3.18005604e+01, 4.24127046e+00, 2.21510494e+01, 1.51818475e+01])[...,None]; # semis
X = results[features].values
X = np.append(np.ones(X.shape[0])[...,None] , X, axis=1)
results['predicted_score'] = np.dot(X, model_coefs)

In [10]:
# Compute ranking
ranking = results[['negative', 'neutral', 'positive', 'tweets', 'predicted_score']].sort_values(by = 'predicted_score', ascending = False)

In [11]:
# set real position - vote of the jury
ranking.loc['POR', 'position_jury'] = 1
ranking.loc['BUL', 'position_jury'] = 2
ranking.loc['SWE', 'position_jury'] = 3
ranking.loc['AUS', 'position_jury'] = 4
ranking.loc['NED', 'position_jury'] = 5
ranking.loc['NOR', 'position_jury'] = 6
ranking.loc['ITA', 'position_jury'] = 7
ranking.loc['MDA', 'position_jury'] = 8
ranking.loc['BEL', 'position_jury'] = 9
ranking.loc['GBR', 'position_jury'] = 10
ranking.loc['AUT', 'position_jury'] = 11
ranking.loc['AZE', 'position_jury'] = 12
ranking.loc['DEN', 'position_jury'] = 13
ranking.loc['ROM', 'position_jury'] = 14
ranking.loc['ARM', 'position_jury'] = 15
ranking.loc['BLR', 'position_jury'] = 16
ranking.loc['HUN', 'position_jury'] = 17
ranking.loc['GRE', 'position_jury'] = 18
ranking.loc['FRA', 'position_jury'] = 19
ranking.loc['CYP', 'position_jury'] = 20
ranking.loc['ISR', 'position_jury'] = 21
ranking.loc['CRO', 'position_jury'] = 22
ranking.loc['POL', 'position_jury'] = 23
ranking.loc['UKR', 'position_jury'] = 24
ranking.loc['GER', 'position_jury'] = 25
ranking.loc['ESP', 'position_jury'] = 26

# set real position - public vote
ranking.loc['POR', 'position_people'] = 1
ranking.loc['BUL', 'position_people'] = 2
ranking.loc['MDA', 'position_people'] = 3
ranking.loc['BEL', 'position_people'] = 4
ranking.loc['ROM', 'position_people'] = 5
ranking.loc['ITA', 'position_people'] = 6
ranking.loc['HUN', 'position_people'] = 7
ranking.loc['SWE', 'position_people'] = 8
ranking.loc['CRO', 'position_people'] = 9
ranking.loc['FRA', 'position_people'] = 10
ranking.loc['AZE', 'position_people'] = 11
ranking.loc['POL', 'position_people'] = 12
ranking.loc['BLR', 'position_people'] = 13
ranking.loc['CYP', 'position_people'] = 14
ranking.loc['NOR', 'position_people'] = 15
ranking.loc['GRE', 'position_people'] = 16
ranking.loc['UKR', 'position_people'] = 17
ranking.loc['ARM', 'position_people'] = 18
ranking.loc['NED', 'position_people'] = 19
ranking.loc['GBR', 'position_people'] = 20
ranking.loc['DEN', 'position_people'] = 21
ranking.loc['ISR', 'position_people'] = 22
ranking.loc['ESP', 'position_people'] = 23
ranking.loc['GER', 'position_people'] = 24
ranking.loc['AUS', 'position_people'] = 25
ranking.loc['AUT', 'position_people'] = 26

# set real position - all
ranking.loc['POR', 'position'] = 1
ranking.loc['BUL', 'position'] = 2
ranking.loc['MDA', 'position'] = 3
ranking.loc['BEL', 'position'] = 4
ranking.loc['SWE', 'position'] = 5
ranking.loc['ITA', 'position'] = 6
ranking.loc['ROM', 'position'] = 7
ranking.loc['HUN', 'position'] = 8
ranking.loc['AUS', 'position'] = 9
ranking.loc['NOR', 'position'] = 10
ranking.loc['NED', 'position'] = 11
ranking.loc['FRA', 'position'] = 12
ranking.loc['CRO', 'position'] = 13
ranking.loc['AZE', 'position'] = 14
ranking.loc['GBR', 'position'] = 15
ranking.loc['AUT', 'position'] = 16
ranking.loc['BLR', 'position'] = 17
ranking.loc['ARM', 'position'] = 18
ranking.loc['GRE', 'position'] = 19
ranking.loc['DEN', 'position'] = 20
ranking.loc['CYP', 'position'] = 21
ranking.loc['POL', 'position'] = 22
ranking.loc['ISR', 'position'] = 23
ranking.loc['UKR', 'position'] = 24
ranking.loc['GER', 'position'] = 25
ranking.loc['ESP', 'position'] = 26

# set predicted position as a column
ranking = ranking.reset_index()
ranking['predicted_position'] = ranking.index + 1

In [12]:
# compute ranking prediction errors
print('RMSE predicted - final:  {}'.format(np.sqrt((1/ranking.shape[0])*np.sum(np.power(ranking['predicted_position'] - ranking['position'], 2))))) 
print('RMSE predicted - jury:   {}'.format(np.sqrt((1/ranking.shape[0])*np.sum(np.power(ranking['predicted_position'] - ranking['position_jury'], 2))))) 
print('RMSE predicted - people: {}'.format(np.sqrt((1/ranking.shape[0])*np.sum(np.power(ranking['predicted_position'] - ranking['position_people'], 2))))) 

RMSE predicted - final:  7.716116998754072
RMSE predicted - jury:   8.348191881617701
RMSE predicted - people: 8.862192817720768


In [13]:
# count top-5 errors
top_n = 3; print('Final Top-3 correct:   {}'.format(sum(ranking.head(top_n)['position'] <= top_n)))
top_n = 5; print('Final Top-5 correct:   {}'.format(sum(ranking.head(top_n)['position'] <= top_n)))
top_n = 10; print('Final Top-10 correct:  {}\n'.format(sum(ranking.head(top_n)['position'] <= top_n)))

top_n = 3; print('Jury Top-3 correct:    {}'.format(sum(ranking.head(top_n)['position_jury'] <= top_n)))
top_n = 5; print('Jury Top-5 correct:    {}'.format(sum(ranking.head(top_n)['position_jury'] <= top_n)))
top_n = 10; print('Jury Top-10 correct:   {}\n'.format(sum(ranking.head(top_n)['position_jury'] <= top_n)))

top_n = 3; print('People Top-3 correct:  {}'.format(sum(ranking.head(top_n)['position_people'] <= top_n)))
top_n = 5; print('People Top-5 correct:  {}'.format(sum(ranking.head(top_n)['position_people'] <= top_n)))
top_n = 10; print('People Top-10 correct: {}\n'.format(sum(ranking.head(top_n)['position_people'] <= top_n)))

bottom_n = 5;
print('Final Bottom-5 correct:  {}'.format(sum(ranking.tail(bottom_n)['position'] <= top_n)))
print('Jury Bottom-5 correct:   {}'.format(sum(ranking.tail(bottom_n)['position_jury'] <= top_n)))
print('People Bottom-5 correct: {}'.format(sum(ranking.tail(bottom_n)['position_people'] <= top_n)))

Final Top-3 correct:   2
Final Top-5 correct:   2
Final Top-10 correct:  6

Jury Top-3 correct:    1
Jury Top-5 correct:    2
Jury Top-10 correct:   7

People Top-3 correct:  2
People Top-5 correct:  2
People Top-10 correct: 7

Final Bottom-5 correct:  2
Jury Bottom-5 correct:   2
People Bottom-5 correct: 1
